In [ ]:

# -*- coding: utf-8 -*-
import pandas
import csv
import re
import time
import json
from urllib.request import urlopen
import urllib
 
# 原数据文件格式： 序号 + 起点纬度 + 起点经度 + 终点纬度 + 终点经度
origin_path = 'E://data/起点终点坐标.xlsx'   # 原始坐标文件路径
result_path = 'E://data/result122901.txt'   # 爬取数据文件保存路径
 
"""# 百度地图提供的api服务网址
"http://api.map.baidu.com/routematrix/v2/driving?output=json"  # 驾车(routematrix 批量算路)
'http://api.map.baidu.com/routematrix/v2/riding?output=json'  # 骑行
'http://api.map.baidu.com/routematrix/v2/walking?output=json'  # 步行
'http://api.map.baidu.com/direction/v2/transit?output=json'  # bus(direction路线规划)
"""
 
# 声明坐标格式,bd09ll(百度经纬度坐标);bd09mc(百度摩卡托坐标);gcj02(国测局加密坐标),wgs84(gps设备获取的坐标)
cod = r"&coord_type=bd09ll"
 
# AK为从百度地图网站申请的秘钥
AK = ['4KQuylnCQEIWYG65tEMQ9spP4FFBvNoI',]
 
dfBase = pandas.read_excel(origin_path, names=['序号','起点纬度','起点经度','终点纬度','终点经度'])
#dfBase.head()
dataList = []  # 储存获取的路线数据
akn = 0  # 使用第几个ak
for i in range(len(dfBase)):
    print(i)
    ak = AK[akn]
    out_lat = dfBase.at[i,'起点纬度']
    out_lng = dfBase.at[i,'起点经度']
    des_lat = dfBase.at[i,'终点纬度']
    des_lng = dfBase.at[i,'终点经度']
    """
    # 获取驾车路径:常规路线规划(不考虑路况) 
    以下是可选参数
    #  tactics =10不走高速;=11常规路线;=12距离较短;=13距离较短
    """
    url_drive = r"http://api.map.baidu.com/routematrix/v2/driving?output=json&origins={0},{1}&destinations={2},{3}&{4}&tactics=11&ak={4}".format(out_lat,out_lng,des_lat,des_lng,ak)
    result_drive = json.loads(urlopen(url_drive).read())  # json转dict
    status_drive = result_drive['status']
    print('ak秘钥：{0}  获取驾车路线状态码status：{1}'.format(ak, status_drive))
    if status_drive == 0:  # 状态码为0：无异常
        distance_drive = result_drive['result'][0]['distance']['value']  # 里程(米)
        timesec_drive = result_drive['result'][0]['duration']['value']  # 耗时(秒)
    elif status_drive == 302 or status_drive == 210 or status_drive == 201:  # 302:额度不足;210:IP验证未通过
        distance_drive = timesec_drive = 'AK错误'
        akn += 1
        ak = AK[akn]
    else:
        distance_drive = timesec_drive = '请求错误'
 
    
    """
    ### 以下是乘车规划
    可选参数
    tac_bus = r'&tactics_incity=0'
    # 市内公交换乘策略 可选，默认为0      0推荐；1少换乘；2少步行；3不坐地铁；4时间短；5地铁优先
    city_bus = r'&tactics_intercity=0'
    # 跨城公交换乘策略  可选，默认为0    0时间短；1出发早；2价格低；
    city_type = r'&trans_type_intercity=0'
    # 跨城交通方式策略  可选，默认为0  0火车优先；1飞机优先；2大巴优先；
    """
    url_bus = r'http://api.map.baidu.com/direction/v2/transit?output=json&origin={0},{1}&destination={2},{3}&{4}&ak={4}'.format(out_lat,out_lng,des_lat,des_lng,ak)
    print(url_bus)
    result_bus = json.loads(urlopen(url_bus).read())
    status_bus = result_bus['status']
    print('ak秘钥：{0}  获取乘车路线状态码status：{1}'.format(ak, status_bus))
    if status_bus == 0:
        rsls = result_bus['result']['routes']
        if rsls == []:    # 无方案时状态也为0，但只返回一个空list
            distance_bus = timesec_bus = cost_bus = '无公交方案'
        else:
            distance_bus = result_bus['result']['routes'][0]['distance']  # 乘车路线距离总长(米)
            timesec_bus = result_bus['result']['routes'][0]['duration']  # 乘车时间(秒)
            cost_bus = result_bus['result']['routes'][0]['price']  # 乘车费用(元)
    elif status_bus == 302 or status_bus == 210 or status_bus == 201:
        distance_bus = timesec_bus = cost_bus = 'AK错误'
        akn += 1
        ak = AK[akn]
    elif status_bus == 1001:
        distance_bus = timesec_bus = cost_bus = '无公交方案'
    else:  # 其他类型状态码(服务器错误)
        distance_bus = timesec_bus = cost_bus = '请求错误'
    
    dataList.append([ak,status_drive,distance_drive,timesec_drive,status_bus,distance_bus,timesec_bus,cost_bus])
 
dfAll = pandas.DataFrame(dataList, columns=['ak','status_drive','distance_drive','timesec_drive','status_bus','distance_bus','timesec_bus','cost_bus'])
dfAll

